In [3]:
!pip install fastai

In [5]:
pip install fastbook

In [6]:
from fastbook import *

In [54]:
urls = search_images_ddg('empty plate photos', max_images=1)
len(urls)
urls[0]

In [52]:
dest = Path('emptyPlate.jpg')
if not dest.exists():
    download_url(urls[0], dest, show_progress=True)

In [53]:
im = Image.open(dest)
im.to_thumb(256, 256)

In [51]:
searches = 'hot dog', 'empty plate'
path = Path('hot_dog_or_not')

if not path.exists():
    for o in searches:
        dest = (path/o)
        dest.mkdir(exist_ok=True, parents=True)
        results = search_images_ddg(f'{o} photo')
        download_images(dest, urls=results[:200])
        resize_images(dest, max_size=400, dest=dest)

In [48]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink);

In [50]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=.2, seed=42),
    get_y = parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

dls.show_batch(max_n=8)

In [18]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

In [55]:
is_hot_dog,_,probs = learn.predict(PILImage.create('emptyPlate.jpg'))
print(f"This is a: {is_hot_dog}.")
print(f"Probability it's not a hot dog: {probs[0]:.4f}")